<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security Workshop
This project is licensed under the MIT License.
See LICENSE file in the root directory for full license text.
-->

*Copyright © 2025 Milin Patel. All Rights Reserved.*

# Notebook 15: Uncertainty Types in Deep Learning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_4_Uncertainty_Estimation_and_Validation/notebooks/15_Uncertainty_Types_in_Deep_Learning.ipynb)

**Session 4: Uncertainty Estimation and Validation**  

## Learning Objectives
- Understand aleatoric vs epistemic uncertainty
- Identify sources of uncertainty in AV perception
- Recognize why uncertainty quantification is critical for safety
- Connect uncertainty to ISO 26262 and ISO 21448 (SOTIF)

---

## Introduction

**Why Uncertainty Matters for Autonomous Vehicles:**

Imagine an AV perception system detecting a pedestrian with 95% confidence. Should the vehicle brake?
- If the model is well-calibrated: Yes, high confidence means high probability.
- If the model is overconfident: Maybe not - the 95% could be meaningless.
- If we know the model is uncertain (epistemic uncertainty is high): We should be cautious.

**Safety standards require uncertainty awareness:**
- **ISO 26262** (functional safety): Requires validation that the system handles uncertainty
- **ISO 21448** (SOTIF): Directly addresses performance limitations and unknown unsafe scenarios

Uncertainty quantification helps us:
1. Detect out-of-distribution scenarios
2. Trigger fallback behaviors
3. Understand model limitations
4. Provide safety evidence

In [ ]:
# Setup
!pip install -q torch torchvision matplotlib seaborn numpy scipy scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)

## 1. Types of Uncertainty

### 1.1 Aleatoric Uncertainty (Data Uncertainty)

**Definition:** Irreducible uncertainty inherent in the data itself.

**Characteristics:**
- Cannot be reduced by collecting more data
- Due to noise in observations
- Also called "statistical uncertainty" or "data uncertainty"

**Sources in AV Perception:**
- **Sensor noise:** Camera noise, LiDAR measurement errors
- **Environmental conditions:** Rain drops on camera, fog, glare
- **Motion blur:** Fast-moving objects
- **Occlusions:** Partially visible pedestrians
- **Label noise:** Inconsistent human annotations

**Two types:**
- **Homoscedastic:** Constant across all inputs (e.g., camera sensor noise)
- **Heteroscedastic:** Varies with input (e.g., more uncertainty for distant objects)

### 1.2 Epistemic Uncertainty (Model Uncertainty)

**Definition:** Reducible uncertainty due to lack of knowledge.

**Characteristics:**
- Can be reduced with more training data
- Due to limited training data or model capacity
- Also called "model uncertainty" or "knowledge uncertainty"

**Sources in AV Perception:**
- **Insufficient training data:** Rare scenarios not well-represented
- **Out-of-distribution inputs:** Novel objects or conditions
- **Model capacity:** Limited representational power
- **Domain shift:** Training in sunny weather, testing in snow

**Key insight for safety:** High epistemic uncertainty signals "I haven't seen this before" → safety-critical for AVs!

## 2. Visualizing Uncertainty Types

Let's create a simple toy problem to visualize both types of uncertainty.

In [ ]:
# Create dataset with gap (high epistemic uncertainty region)
def create_dataset_with_gap():
    """Create 1D regression dataset with a gap to simulate OOD region."""
    # Training data: two separate regions with gap
    x_train_1 = np.linspace(-4, -1, 30)
    x_train_2 = np.linspace(1, 4, 30)
    x_train = np.concatenate([x_train_1, x_train_2])
    
    # True function
    y_true = np.sin(x_train * 2)
    
    # Add aleatoric noise (heteroscedastic - more noise for larger |x|)
    noise_std = 0.1 + 0.05 * np.abs(x_train)
    y_train = y_true + np.random.randn(len(x_train)) * noise_std
    
    # Test data: includes the gap region
    x_test = np.linspace(-5, 5, 200)
    
    return x_train, y_train, x_test, noise_std

x_train, y_train, x_test, noise_std = create_dataset_with_gap()

# Visualize
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(x_train, y_train, alpha=0.6, s=50, label='Training data')
plt.axvspan(-1, 1, alpha=0.2, color='red', label='Gap (high epistemic uncertainty)')
plt.xlabel('Input x')
plt.ylabel('Output y')
plt.title('Dataset with Gap (OOD Region)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Show heteroscedastic aleatoric uncertainty
for i in range(0, len(x_train), 5):
    plt.errorbar(x_train[i], y_train[i], yerr=noise_std[i]*2, 
                 fmt='o', alpha=0.5, capsize=3)
plt.xlabel('Input x')
plt.ylabel('Output y')
plt.title('Aleatoric Uncertainty (Heteroscedastic)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Dataset characteristics:")
print(f"- Training samples: {len(x_train)}")
print(f"- Gap region: [-1, 1] (no training data)")
print(f"- Aleatoric noise: heteroscedastic (varies with |x|)")

In [ ]:
# Simple neural network for regression
class SimpleNN(nn.Module):
    def __init__(self, hidden_size=50):
        super().__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Train the model
model = SimpleNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Convert to tensors
X_train_tensor = torch.FloatTensor(x_train.reshape(-1, 1))
y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1))

# Training loop
model.train()
losses = []
for epoch in range(1000):
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}/1000, Loss: {loss.item():.4f}")

# Plot training loss
plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Get predictions (single model - no uncertainty yet)
model.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(x_test.reshape(-1, 1))
    y_pred = model(X_test_tensor).numpy().flatten()

# Visualize predictions
plt.figure(figsize=(12, 5))
plt.scatter(x_train, y_train, alpha=0.6, s=50, label='Training data', c='blue')
plt.plot(x_test, y_pred, 'r-', linewidth=2, label='Model prediction')
plt.axvspan(-1, 1, alpha=0.2, color='red', label='Gap (OOD region)')
plt.xlabel('Input x')
plt.ylabel('Output y')
plt.title('Single Model Prediction (No Uncertainty Information!)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("⚠️ Problem: The model makes confident predictions in the gap region!")
print("Without uncertainty estimation, we don't know the model is unreliable there.")

## 3. Sources of Uncertainty in AV Perception

### 3.1 Sensor-Level Uncertainty (Aleatoric)

**Camera:**
- Photon noise (especially in low light)
- Motion blur from fast movement
- Lens distortion at edges
- Rain drops, dirt, glare

**LiDAR:**
- Range measurement noise (~2-3 cm for modern LiDAR)
- Reduced returns on dark/wet surfaces
- Interference from other LiDAR
- Sparse point clouds for distant objects

**Radar:**
- Angular resolution limitations
- Multipath reflections
- Ghost targets

### 3.2 Perception-Level Uncertainty (Both Types)

**Aleatoric:**
- Occlusions (pedestrian behind car)
- Truncated objects (only part visible)
- Ambiguous situations (is it a bag or animal?)

**Epistemic:**
- Novel objects not in training data
- Rare scenarios (e.g., overturned truck)
- Domain shift (new city, new weather)
- Corner cases

### 3.3 Simulation of AV Perception Uncertainties

In [ ]:
# Simulate different uncertainty sources
def simulate_perception_uncertainty():
    """Simulate various uncertainty sources in AV perception."""
    
    scenarios = [
        {
            'name': 'Clear Weather, Well-Lit',
            'aleatoric': 0.02,  # Low sensor noise
            'epistemic': 0.01,  # Well-trained scenario
            'color': 'green'
        },
        {
            'name': 'Rain, Nighttime',
            'aleatoric': 0.15,  # High sensor noise
            'epistemic': 0.05,  # Still trained scenario
            'color': 'yellow'
        },
        {
            'name': 'Novel Object (Construction)',
            'aleatoric': 0.03,  # Normal sensor noise
            'epistemic': 0.25,  # Not in training data!
            'color': 'orange'
        },
        {
            'name': 'Severe Occlusion',
            'aleatoric': 0.20,  # Missing information
            'epistemic': 0.08,  # Trained but challenging
            'color': 'orange'
        },
        {
            'name': 'Unknown Scenario (Snow)',
            'aleatoric': 0.12,  # Moderate sensor issues
            'epistemic': 0.35,  # Never seen snow!
            'color': 'red'
        }
    ]
    
    # Visualize
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Bar chart
    names = [s['name'] for s in scenarios]
    aleatoric = [s['aleatoric'] for s in scenarios]
    epistemic = [s['epistemic'] for s in scenarios]
    colors = [s['color'] for s in scenarios]
    
    x = np.arange(len(names))
    width = 0.35
    
    ax1.bar(x - width/2, aleatoric, width, label='Aleatoric (Data)', alpha=0.8)
    ax1.bar(x + width/2, epistemic, width, label='Epistemic (Model)', alpha=0.8)
    ax1.set_ylabel('Uncertainty Level')
    ax1.set_title('Uncertainty Sources in Different AV Scenarios')
    ax1.set_xticks(x)
    ax1.set_xticklabels(names, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Scatter plot: total uncertainty
    total_uncertainty = [a + e for a, e in zip(aleatoric, epistemic)]
    for i, (a, e, name, color) in enumerate(zip(aleatoric, epistemic, names, colors)):
        ax2.scatter(a, e, s=500, c=color, alpha=0.6, edgecolors='black', linewidth=2)
        ax2.annotate(str(i+1), (a, e), ha='center', va='center', fontweight='bold')
    
    ax2.set_xlabel('Aleatoric Uncertainty (Irreducible)')
    ax2.set_ylabel('Epistemic Uncertainty (Reducible)')
    ax2.set_title('Uncertainty Space Map')
    ax2.grid(True, alpha=0.3)
    
    # Add diagonal lines for total uncertainty
    for total in [0.1, 0.2, 0.3, 0.4]:
        x_line = np.linspace(0, total, 100)
        y_line = total - x_line
        ax2.plot(x_line, y_line, 'k--', alpha=0.2)
        ax2.text(total/2, total/2, f'Total={total:.1f}', 
                rotation=-45, alpha=0.5, fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Print insights
    print("\n🎯 Key Insights:")
    print("\n1. GREEN scenarios: Low total uncertainty → Safe to operate")
    print("2. YELLOW scenarios: Moderate uncertainty → Operate with caution")
    print("3. ORANGE scenarios: High uncertainty → Consider fallback")
    print("4. RED scenarios: Very high epistemic uncertainty → STOP or minimal risk condition!")
    print("\n5. High EPISTEMIC uncertainty = 'Never seen this before' → Most dangerous!")
    print("6. High ALEATORIC uncertainty = 'Noisy data' → Can handle with robust design")
    
    return scenarios

scenarios = simulate_perception_uncertainty()

## 4. Example: Pedestrian Detection with Uncertainty

Let's simulate a pedestrian detection scenario with both uncertainty types.

In [ ]:
def pedestrian_detection_uncertainty_example():
    """
    Simulate pedestrian detection with different uncertainty conditions.
    """
    
    cases = [
        {
            'scenario': 'Clear view, daytime',
            'confidence': 0.95,
            'aleatoric_std': 0.02,
            'epistemic_std': 0.01,
            'decision': 'High confidence - Normal operation',
            'safety_level': 'Safe'
        },
        {
            'scenario': 'Partial occlusion',
            'confidence': 0.78,
            'aleatoric_std': 0.15,
            'epistemic_std': 0.05,
            'decision': 'Moderate confidence - Monitor closely',
            'safety_level': 'Caution'
        },
        {
            'scenario': 'Person in unusual pose',
            'confidence': 0.72,
            'aleatoric_std': 0.05,
            'epistemic_std': 0.20,
            'decision': 'High epistemic uncertainty - Slow down',
            'safety_level': 'Warning'
        },
        {
            'scenario': 'Person-like object (statue)',
            'confidence': 0.65,
            'aleatoric_std': 0.08,
            'epistemic_std': 0.25,
            'decision': 'Very uncertain - Conservative action',
            'safety_level': 'Alert'
        },
        {
            'scenario': 'Night + rain + occlusion',
            'confidence': 0.55,
            'aleatoric_std': 0.25,
            'epistemic_std': 0.15,
            'decision': 'High total uncertainty - Reduce speed significantly',
            'safety_level': 'Danger'
        }
    ]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, case in enumerate(cases):
        ax = axes[i]
        
        # Simulate detection distribution
        confidence = case['confidence']
        total_std = np.sqrt(case['aleatoric_std']**2 + case['epistemic_std']**2)
        
        # Create confidence distribution
        x = np.linspace(0, 1, 200)
        
        # Aleatoric component (narrow, fixed)
        aleatoric_dist = np.exp(-0.5 * ((x - confidence) / case['aleatoric_std'])**2)
        aleatoric_dist /= (case['aleatoric_std'] * np.sqrt(2 * np.pi))
        
        # Total uncertainty (wider)
        total_dist = np.exp(-0.5 * ((x - confidence) / total_std)**2)
        total_dist /= (total_std * np.sqrt(2 * np.pi))
        
        # Plot
        ax.fill_between(x, 0, total_dist, alpha=0.3, color='red', 
                        label=f'Total Unc. (σ={total_std:.3f})')
        ax.fill_between(x, 0, aleatoric_dist, alpha=0.5, color='blue',
                       label=f'Aleatoric (σ={case["aleatoric_std"]:.3f})')
        ax.axvline(confidence, color='black', linestyle='--', linewidth=2, 
                  label=f'Conf={confidence:.2f}')
        
        # Decision threshold
        ax.axvline(0.8, color='green', linestyle=':', linewidth=1, alpha=0.5)
        ax.text(0.8, ax.get_ylim()[1]*0.9, 'Decision\nThreshold', 
               ha='center', fontsize=8, color='green')
        
        # Safety color coding
        color_map = {
            'Safe': 'lightgreen',
            'Caution': 'yellow',
            'Warning': 'orange',
            'Alert': 'orangered',
            'Danger': 'red'
        }
        ax.set_facecolor(color_map[case['safety_level']])
        ax.set_alpha(0.1)
        
        ax.set_xlabel('Confidence')
        ax.set_ylabel('Probability Density')
        ax.set_title(f"{case['scenario']}\n{case['safety_level']}", fontsize=10, fontweight='bold')
        ax.legend(fontsize=8, loc='upper left')
        ax.grid(True, alpha=0.3)
        ax.set_xlim([0, 1])
    
    # Remove extra subplot
    axes[-1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print decision table
    print("\n" + "="*80)
    print("PEDESTRIAN DETECTION UNCERTAINTY ANALYSIS")
    print("="*80)
    print(f"\n{'Scenario':<30} {'Conf':<6} {'Aleat':<6} {'Epist':<6} {'Total':<6} {'Decision':<40}")
    print("-"*80)
    
    for case in cases:
        total = np.sqrt(case['aleatoric_std']**2 + case['epistemic_std']**2)
        print(f"{case['scenario']:<30} {case['confidence']:<6.2f} "
              f"{case['aleatoric_std']:<6.3f} {case['epistemic_std']:<6.3f} "
              f"{total:<6.3f} {case['decision']:<40}")
    
    print("\n" + "="*80)
    print("\n🔑 Decision Logic:")
    print("1. High confidence + Low epistemic uncertainty → Normal operation")
    print("2. Moderate confidence OR Moderate epistemic → Caution, monitor")
    print("3. Low confidence OR High epistemic → Slow down, conservative")
    print("4. Very high total uncertainty → Significant speed reduction or stop")
    print("\n⚠️  Epistemic uncertainty is MORE critical than aleatoric for safety decisions!")

pedestrian_detection_uncertainty_example()

## 5. Why Uncertainty Matters for Safety Standards

### 5.1 ISO 26262 (Functional Safety)

**Relevant sections:**
- **Part 6:** Product development at the software level
- **Part 8:** Supporting processes (validation)

**Requirements:**
- System must handle internal failures and random hardware failures
- Software must be validated for specified operating conditions
- **Uncertainty quantification helps:**
  - Detect when system is operating outside validated conditions
  - Trigger fail-safe mechanisms
  - Provide diagnostic coverage

### 5.2 ISO 21448 (SOTIF - Safety of the Intended Functionality)

**Focus:** Performance limitations and unknown unsafe scenarios

**Key concepts:**
1. **Known safe scenarios:** Where system performs correctly
2. **Known unsafe scenarios:** Where system has known limitations
3. **Unknown unsafe scenarios:** Not yet discovered (biggest risk!)
4. **Unknown safe scenarios:** System works but not validated

**How uncertainty helps:**
- **High epistemic uncertainty → Likely unknown scenario!**
- Can detect transition from known to unknown scenarios
- Enables runtime monitoring
- Helps define Operational Design Domain (ODD)

### 5.3 Uncertainty-Aware Decision Making

In [ ]:
def uncertainty_aware_decision_logic():
    """
    Demonstrate uncertainty-aware decision logic for AVs.
    """
    
    # Define decision zones
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Create grid
    aleatoric_range = np.linspace(0, 0.3, 100)
    epistemic_range = np.linspace(0, 0.4, 100)
    A, E = np.meshgrid(aleatoric_range, epistemic_range)
    
    # Decision zones based on total uncertainty and epistemic weight
    total_uncertainty = np.sqrt(A**2 + E**2)
    epistemic_weight = E / (total_uncertainty + 1e-6)
    
    # Zone 1: Normal operation (green)
    # Zone 2: Caution (yellow)
    # Zone 3: Fallback (orange)
    # Zone 4: Minimal risk condition (red)
    
    zones = np.zeros_like(total_uncertainty)
    zones[(total_uncertainty < 0.05)] = 1  # Normal
    zones[(total_uncertainty >= 0.05) & (total_uncertainty < 0.15) & (E < 0.1)] = 2  # Caution
    zones[(total_uncertainty >= 0.05) & (total_uncertainty < 0.15) & (E >= 0.1)] = 3  # Fallback
    zones[(total_uncertainty >= 0.15) & (E < 0.15)] = 3  # Fallback
    zones[(total_uncertainty >= 0.15) & (E >= 0.15)] = 4  # Minimal risk
    zones[(E >= 0.25)] = 4  # High epistemic always triggers minimal risk
    
    # Plot zones
    colors = ['white', 'lightgreen', 'yellow', 'orange', 'red']
    labels = ['', 'Normal Operation', 'Caution Mode', 'Fallback', 'Minimal Risk Condition']
    
    contour = ax.contourf(A, E, zones, levels=[0, 1, 2, 3, 4, 5], 
                          colors=colors[1:], alpha=0.6)
    
    # Add contour lines
    contour_lines = ax.contour(A, E, zones, levels=[1, 2, 3, 4], 
                               colors='black', linewidths=2)
    
    # Plot example scenarios
    example_points = [
        (0.02, 0.01, 'Clear day', 'o', 'blue'),
        (0.15, 0.05, 'Rain', 's', 'cyan'),
        (0.05, 0.20, 'Novel object', '^', 'purple'),
        (0.20, 0.15, 'Heavy occlusion', 'D', 'brown'),
        (0.12, 0.30, 'Unknown scenario', 'X', 'black')
    ]
    
    for aleat, epist, label, marker, color in example_points:
        ax.scatter(aleat, epist, s=200, marker=marker, c=color, 
                  edgecolors='black', linewidth=2, label=label, zorder=5)
    
    # Add epistemic threshold line
    ax.axhline(y=0.25, color='darkred', linestyle='--', linewidth=2, 
              label='Epistemic threshold')
    
    ax.set_xlabel('Aleatoric Uncertainty (Irreducible)', fontsize=12)
    ax.set_ylabel('Epistemic Uncertainty (Reducible)', fontsize=12)
    ax.set_title('Uncertainty-Aware Decision Zones for AV Control\n(ISO 21448 SOTIF Compliant)', 
                fontsize=14, fontweight='bold')
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Add zone labels
    ax.text(0.02, 0.02, 'NORMAL\nOPERATION', fontsize=10, fontweight='bold', 
           ha='center', va='center', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(0.20, 0.03, 'CAUTION', fontsize=10, fontweight='bold',
           ha='center', va='center', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(0.10, 0.12, 'FALLBACK', fontsize=10, fontweight='bold',
           ha='center', va='center', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(0.20, 0.30, 'MINIMAL RISK\nCONDITION', fontsize=10, fontweight='bold',
           ha='center', va='center', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print("\n🚗 Uncertainty-Aware AV Decision Logic:")
    print("="*70)
    print("\nZone 1 - NORMAL OPERATION:")
    print("  • Low total uncertainty (<0.05)")
    print("  • Low epistemic uncertainty")
    print("  → Action: Normal driving, full functionality")
    print("\nZone 2 - CAUTION MODE:")
    print("  • Moderate aleatoric uncertainty")
    print("  • Still low epistemic uncertainty")
    print("  → Action: Reduce speed slightly, increase safety margins")
    print("\nZone 3 - FALLBACK:")
    print("  • Moderate epistemic uncertainty OR high total uncertainty")
    print("  • System entering unfamiliar territory")
    print("  → Action: Activate fallback system, request human takeover")
    print("\nZone 4 - MINIMAL RISK CONDITION:")
    print("  • High epistemic uncertainty (>0.25)")
    print("  • System in unknown scenario (SOTIF critical!)")
    print("  → Action: Safe stop, minimal risk maneuver, do NOT continue")
    print("\n⚠️  Key principle: Epistemic uncertainty is weighted MORE heavily!")
    print("   'Unknown unknowns' are more dangerous than 'known noise'")

uncertainty_aware_decision_logic()

## 6. Uncertainty Quantification Methods Overview

We'll explore these in detail in the next notebooks:

### 6.1 Bayesian Methods
- **Bayesian Neural Networks (BNNs):** Full posterior over weights
- **Monte Carlo Dropout:** Approximate Bayesian inference
- **Variational Inference:** Approximate posterior

**Captures:** Primarily epistemic uncertainty

### 6.2 Ensemble Methods
- **Deep Ensembles:** Multiple independent models
- **Snapshot Ensembles:** Single training trajectory
- **Batch Ensembles:** Parameter-efficient ensembles

**Captures:** Both aleatoric and epistemic uncertainty

### 6.3 Single-Model Methods
- **Evidential Deep Learning:** Direct uncertainty prediction
- **Heteroscedastic Networks:** Learn aleatoric uncertainty
- **Deterministic Uncertainty Quantification (DUQ)**

**Captures:** Can separate both types

### 6.4 Post-hoc Methods
- **Temperature Scaling:** Calibrate confidences
- **Platt Scaling**
- **Isotonic Regression**

**Purpose:** Improve calibration of existing models

## Summary and Key Takeaways

### What We Learned

1. **Two Types of Uncertainty:**
   - **Aleatoric:** Irreducible noise in data (sensor noise, occlusions)
   - **Epistemic:** Reducible model uncertainty (lack of training data, OOD)

2. **Sources in AV Perception:**
   - Sensor-level: Camera noise, LiDAR errors, weather effects
   - Perception-level: Occlusions, novel objects, domain shift

3. **Safety Implications:**
   - **ISO 26262:** Uncertainty helps detect operating outside validated conditions
   - **ISO 21448 (SOTIF):** High epistemic uncertainty signals unknown scenarios

4. **Decision Making:**
   - Epistemic uncertainty should be weighted MORE heavily
   - Unknown scenarios are more dangerous than noisy data
   - Enables runtime monitoring and fallback triggers

### Next Steps

In the following notebooks, we'll learn:
- **Notebook 16:** How to implement MC Dropout and Ensembles
- **Notebook 17:** How to calibrate model confidence
- **Notebook 18:** How to validate safety of ML-based systems

---

## Interactive Exercise

**Try this:**
1. Modify the pedestrian detection scenarios - add your own cases
2. Adjust the decision zone thresholds - make them more/less conservative
3. Think about: What uncertainty threshold would YOU use for your AV?

**Discussion question:**
- An AV detects a pedestrian with 90% confidence but high epistemic uncertainty.
- Should it brake? Why or why not?
- How does this relate to SOTIF?